### POS

In [2]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score,confusion_matrix
from spacy.tokenizer import Tokenizer
from spacy.lang.char_classes import ALPHA, ALPHA_LOWER, ALPHA_UPPER, CONCAT_QUOTES, LIST_ELLIPSES, LIST_ICONS
from spacy.util import compile_infix_regex
import spacy
import ast
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import plotly.express as px
import plotly.graph_objects as go
# from google.colab import drive

In [ ]:
# !pip install plotly
# !pip install pandas
# !pip install spacy
# !pip install tqdm
!pip install matplotlib, seaborn, collections, ast, sklearn

In [2]:
# drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("C:/Users/asus/OneDrive/Desktop/NLP assignment/NLP_task_2.csv")
df.head()

,Unnamed: 0,category,text
0,0,tech,tv future in the hands of viewers with home th...
1,1,business,worldcom boss left books alone former worldc...
2,2,sport,tigers wary of farrell gamble leicester say ...
3,3,sport,yeading face newcastle in fa cup premiership s...
4,4,entertainment,ocean s twelve raids box office ocean s twelve...


In [5]:
# Load a spaCy model
nlp = spacy.load("en_core_web_sm", disable=["parser", "lemmatizer"])

pos_list = []

for i in tqdm(range(len(df))):
    # Convert string representation of list back to list
    words = ast.literal_eval(df.text.iloc[i])

    # Use spaCy to create a Doc from the list of words (tokens)
    doc = spacy.tokens.Doc(nlp.vocab, words=words)

    # Apply the pipeline's components except tokenizer (since we already have tokens)
    for name, proc in nlp.pipeline:
        doc = proc(doc)

    # Extract POS tags
    pos_tags = [token.tag_ for token in doc]
    pos_list.append(pos_tags)

# Update DataFrame with the new columns
df['predicted_POS'] = pos_list

  0%|          | 0/30 [00:00<?, ?it/s]


SyntaxError: invalid syntax (<unknown>, line 1)

  0%|          | 0/30 [00:00<?, ?it/s]


SyntaxError: invalid syntax (<unknown>, line 1)

In [ ]:
def parse_column(entry):
    # Split the entry to get the list elements
    elements = entry.split(']')[0].split('[')[1].split(', ')
    # Strip single and double quotes from each element
    stripped_elements = [element.strip("'\"") for element in elements]
    return stripped_elements

df['Word'] = df['Word'].apply(lambda x: parse_column(x))
df['POS'] = df['POS'].apply(lambda x: parse_column(x))
df['Tag'] = df['Tag'].apply(lambda x: parse_column(x))

In [ ]:
pos = df['POS'].astype('str')
predicted_pos = df['predicted_POS'].astype('str')

In [ ]:
print(accuracy_score(pos, predicted_pos))

In [ ]:
# Flatten the lists
y_true_pos = [tag for sublist in df['POS'] for tag in sublist]
y_pred_pos = [tag for sublist in df['predicted_POS'] for tag in sublist]

In [ ]:
print(accuracy_score(y_true_pos,y_pred_pos))

In [ ]:
# Identify all unique labels
all_labels = sorted(set(y_true_pos + y_pred_pos))

# Count the support for each label in y_true
support_counts = {label: y_true.count(label) for label in all_labels}

# Filter out labels with zero support
labels_with_support = [label for label, count in support_counts.items() if count > 0]

# Filter y_true and y_pred to include only instances with labels having non-zero support
y_true_pos_filtered = [y_true[i] for i in range(len(y_true_pos)) if y_true[i] in labels_with_support]
y_pred_pos_filtered = [y_pred[i] for i in range(len(y_true_pos)) if y_true[i] in labels_with_support]

# Generate the classification report for filtered lists
report = classification_report(y_true_pos_filtered, y_pred_pos_filtered, labels=labels_with_support, zero_division=0)

print(report)


In [ ]:
# Compute confusion matrix
cm = confusion_matrix(y_true_pos_filtered, y_pred_pos_filtered, labels=labels_with_support)

# Extract labels from the data
labels = labels_with_support

# Create a heatmap using Plotly Graph Objects
fig = go.Figure(data=go.Heatmap(
    z=cm,
    x=labels,
    y=labels,
    colorscale='Blues',
    showscale=True
))

fig.update_layout(
    title='Confusion Matrix',
    xaxis_title='Predicted Label',
    yaxis_title='True Label',
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(len(labels))),
        ticktext=labels
    ),
    yaxis=dict(
        tickmode='array',
        tickvals=list(range(len(labels))),
        ticktext=labels,
        tickangle=-45  # Rotate labels
    ),
    autosize=False,
    width=800,  # Adjust width
    height=800,  # Adjust height to accommodate rotated labels
    margin=dict(l=200, r=50, t=50, b=100)  # Adjust margins (left, right, top, bottom)
)

fig.show()


### NER

In [8]:
tag_map = {
  "B-PERSON": "B-per", "I-PERSON": "I-per",
  "B-NORP": "B-org", "I-NORP": "I-org",
  "B-FAC": "B-geo", "I-FAC": "I-geo",
  "B-ORG": "B-org", "I-ORG": "I-org",
  "B-GPE": "B-gpe", "I-GPE": "I-gpe",
  "B-LOC": "B-geo", "I-LOC": "I-geo",
  "B-PRODUCT": "B-art", "I-PRODUCT": "I-art",
  "B-EVENT": "B-eve", "I-EVENT": "I-eve",
  "B-WORK_OF_ART": "B-art", "I-WORK_OF_ART": "I-art",
  "B-LAW": "B-art", "I-LAW": "I-art",
  "B-LANGUAGE": "B-org", "I-LANGUAGE": "I-org",
  "B-DATE": "B-tim", "I-DATE": "I-tim",
  "B-TIME": "B-tim", "I-TIME": "I-tim",
  "B-QUANTITY": "B-num", "I-QUANTITY": "I-num",
  "B-ORDINAL": "B-num", "I-ORDINAL": "I-num",
  "B-CARDINAL": "B-num", "I-CARDINAL": "I-num",
  "B-MONEY": "B-money", "I-MONEY": "I-money",
  "B-PERCENT": "B-percent", "I-PERCENT": "I-percent"
}


In [10]:
#https://spacy.io/usage/linguistic-features#native-tokenizer-additions
##Testing token mismatches
def custom_tokenizer(nlp):
    infixes = (
        LIST_ELLIPSES
        + LIST_ICONS
        + [
            r"(?<=[0-9])[+\-\*^](?=[0-9-])",
            r"(?<=[{al}{q}])\.(?=[{au}{q}])".format(
                al=ALPHA_LOWER, au=ALPHA_UPPER, q=CONCAT_QUOTES
            ),
            r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
            r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
        ]
    )

    infix_re = compile_infix_regex(infixes)

    return Tokenizer(nlp.vocab, prefix_search=nlp.tokenizer.prefix_search,
                                suffix_search=nlp.tokenizer.suffix_search,
                                infix_finditer=infix_re.finditer,
                                token_match=nlp.tokenizer.token_match,
                                rules=nlp.Defaults.tokenizer_exceptions)
nlp.tokenizer = custom_tokenizer(nlp)

ner_list = []
indices_to_drop = []

for i in tqdm(range(len(df))):
  tag = ast.literal_eval(df.Tag.iloc[i])
  word = ast.literal_eval(df.Word.iloc[i])

  sentence = ' '.join(word)

  # Process the reconstructed sentence with spaCy
  doc = nlp(sentence)
  l=[]
  for x in doc:
    if x.ent_type_:
      # l.append(f"{x.ent_iob_}-{x.ent_type_.lower()[0:3]}")
      predicted_tag = f"{x.ent_iob_}-{x.ent_type_}"
      l.append(tag_map[predicted_tag])
    else:
      l.append(f"{x.ent_iob_}")

  if len(l) != len(tag):
    indices_to_drop.append(i)
  else:
    ner_list.append(l)

  0%|          | 0/30 [00:00<?, ?it/s]


SyntaxError: invalid syntax (<unknown>, line 1)

In [ ]:
# Custom function to parse columns based on the provided method
def parse_column(entry):
    # Split the entry to get the list elements
    elements = entry.split(']')[0].split('[')[1].split(', ')
    # Strip single and double quotes from each element
    stripped_elements = [element.strip("'\"") for element in elements]
    return stripped_elements
    # return entry.split(']')[0].split('[')[1].split(', ')

# Apply the custom parsing function to each column (Word, POS, Tag)
if type(df['Word'].iloc[0])!=type([]):
  df['Word'] = df['Word'].apply(lambda x: parse_column(x))
  df['POS'] = df['POS'].apply(lambda x: parse_column(x))
  df['Tag'] = df['Tag'].apply(lambda x: parse_column(x))

# Calculate sentence lengths
df['Sentence_Length'] = df['Word'].apply(len)

In [ ]:
after_removing_df = df.drop(indices_to_drop).reset_index(drop=True)
after_removing_df['ner_predicted'] = ner_list

In [ ]:
# Frequency distribution of POS tags
pos_tags = [pos for sublist in df['POS'] for pos in sublist]
pos_tag_counts = Counter(pos_tags)

# Frequency distribution of POS tags
pos_tags_after = [pos for sublist in after_removing_df['POS'] for pos in sublist]
pos_tag_counts_after = Counter(pos_tags_after)

# Frequency distribution of NER tags
ner_tags = [tag for sublist in df['Tag'] for tag in sublist]
ner_tag_counts = Counter(ner_tags)

# Frequency distribution of NER tags
ner_tags_after = [tag for sublist in after_removing_df['Tag'] for tag in sublist]
ner_tag_counts_after = Counter(ner_tags_after)

In [ ]:
# Plot the distribution of sentence lengths before removal with Plotly
fig_before = px.histogram(df, x='Sentence_Length', nbins=30, title='Distribution of Sentence Lengths Before Removal')
fig_before.update_layout(xaxis_title='Sentence Length', yaxis_title='Frequency', bargap=0.2)
fig_before.show()

# Plot the distribution of sentence lengths after removal with Plotly
fig_after = px.histogram(after_removing_df, x='Sentence_Length', nbins=30, title='Distribution of Sentence Lengths After Removal')
fig_after.update_layout(xaxis_title='Sentence Length', yaxis_title='Frequency', bargap=0.2)
fig_after.show()


In [ ]:
# Assuming pos_tag_counts is available
fig_pos = px.bar(x=list(pos_tag_counts.keys()), y=list(pos_tag_counts.values()), labels={'x': 'POS Tag', 'y': 'Frequency'}, title='Frequency Distribution of POS Tags')
fig_pos.update_layout(xaxis_tickangle=-45)
fig_pos.show()

# For after removal, calculate POS tag counts again using after_removing_df
# This assumes you've recalculated pos_tag_counts for the after_removing_df
fig_pos_after = px.bar(x=list(pos_tag_counts_after.keys()), y=list(pos_tag_counts_after.values()), labels={'x': 'POS Tag', 'y': 'Frequency'}, title='Frequency Distribution of POS Tags After Removal')
fig_pos_after.update_layout(xaxis_tickangle=-45)
fig_pos_after.show()


In [ ]:
fig_ner = px.bar(x=list(ner_tag_counts.keys()), y=list(ner_tag_counts.values()), labels={'x': 'NER Tag', 'y': 'Frequency'}, title='Frequency Distribution of NER Tags')
fig_ner.update_layout(xaxis_tickangle=-45)
fig_ner.show()

# For after removal, calculate NER tag counts again using after_removing_df
# This assumes you've recalculated ner_tag_counts for the after_removing_df
fig_ner_after = px.bar(x=list(ner_tag_counts_after.keys()), y=list(ner_tag_counts_after.values()), labels={'x': 'NER Tag', 'y': 'Frequency'}, title='Frequency Distribution of NER Tags After Removal')
fig_ner_after.update_layout(xaxis_tickangle=-45)
fig_ner_after.show()


In [ ]:
print(ner_tag_counts)
print(ner_tag_counts_after)

In [ ]:
print(pos_tag_counts)
print(pos_tag_counts_after)

In [ ]:


# Plot the distribution of sentence lengths before removal
plt.figure(figsize=(10, 6))
sns.histplot(df['Sentence_Length'], bins=30, kde=True)
plt.title('Distribution of Sentence Lengths Before Removal')
plt.xlabel('Sentence Length')
plt.ylabel('Frequency')
plt.show()



# Plot the distribution of sentence lengths before removal
plt.figure(figsize=(10, 6))
sns.histplot(after_removing_df['Sentence_Length'], bins=30, kde=True)
plt.title('Distribution of Sentence Lengths After Removal')
plt.xlabel('Sentence Length')
plt.ylabel('Frequency')
plt.show()

In [ ]:
y_true = [tag for sublist in after_removing_df['Tag'] for tag in sublist]
y_pred = [tag for sublist in after_removing_df['ner_predicted'] for tag in sublist]

In [ ]:
print(accuracy_score(y_true,y_pred))

In [ ]:
print(classification_report(y_true,y_pred))

In [ ]:
# Identify all unique labels
all_labels = sorted(set(y_true + y_pred))

# Count the support for each label in y_true
support_counts = {label: y_true.count(label) for label in all_labels}

# Filter out labels with zero support
labels_with_support = [label for label, count in support_counts.items() if count > 0]

# Filter y_true and y_pred to include only instances with labels having non-zero support
y_true_filtered = [y_true[i] for i in range(len(y_true)) if y_true[i] in labels_with_support]
y_pred_filtered = [y_pred[i] for i in range(len(y_pred)) if y_true[i] in labels_with_support]

# Generate the classification report for filtered lists
report = classification_report(y_true_filtered, y_pred_filtered, labels=labels_with_support, zero_division=0)

print(report)


In [ ]:
# PERSON:      People, including fictional.
# NORP:        Nationalities or religious or political groups.
# FAC:         Buildings, airports, highways, bridges, etc.
# ORG:         Companies, agencies, institutions, etc.
# GPE:         Countries, cities, states.
# LOC:         Non-GPE locations, mountain ranges, bodies of water.
# PRODUCT:     Objects, vehicles, foods, etc. (Not services.)
# EVENT:       Named hurricanes, battles, wars, sports events, etc.
# WORK_OF_ART: Titles of books, songs, etc.
# LAW:         Named documents made into laws.
# LANGUAGE:    Any named language.
# DATE:        Absolute or relative dates or periods.
# TIME:        Times smaller than a day.
# PERCENT:     Percentage, including ”%“.
# MONEY:       Monetary values, including unit.
# QUANTITY:    Measurements, as of weight or distance.
# ORDINAL:     “first”, “second”, etc.
# CARDINAL:    Numerals that do not fall under another type.

# geo = Geographical Entity
# org = Organization
# per = Person
# gpe = Geopolitical Entity
# tim = Time indicator
# art = Artifact
# eve = Event
# nat = Natural Phenomenon



In [ ]:
# Compute confusion matrix
cm = confusion_matrix(y_true_filtered, y_pred_filtered, labels=labels_with_support)

# Extract labels from the data
labels = labels_with_support

# Create a heatmap using Plotly Graph Objects
fig = go.Figure(data=go.Heatmap(
    z=cm,
    x=labels,
    y=labels,
    colorscale='Blues',
    showscale=True
))

fig.update_layout(
    title='Confusion Matrix',
    xaxis_title='Predicted Label',
    yaxis_title='True Label',
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(len(labels))),
        ticktext=labels
    ),
    yaxis=dict(
        tickmode='array',
        tickvals=list(range(len(labels))),
        ticktext=labels,
        tickangle=-45  # Rotate labels
    ),
    autosize=False,
    width=800,  # Adjust width
    height=800,  # Adjust height to accommodate rotated labels
    margin=dict(l=200, r=50, t=50, b=100)  # Adjust margins (left, right, top, bottom)
)

fig.show()


In [ ]:
# Create a DataFrame from the confusion matrix and labels
conf_df = pd.DataFrame(cm, index=labels, columns=labels)

# Set up the plot
plt.figure(figsize=(12, 10))
sns.set(font_scale=1.2)

# Plot the heatmap with a custom color map
heatmap = sns.heatmap(conf_df, annot=True, fmt='d', cmap="Greys", cbar=False, linewidths=.5)

# Set labels and title
heatmap.set_xlabel('Predicted Labels', fontsize=16)
heatmap.set_ylabel('Actual Labels', fontsize=16)
heatmap.set_title('Confusion Matrix', fontsize=18)

# Rotate the tick labels for better readability
plt.xticks(rotation=45)
plt.yticks(rotation=0)

# Show the plot
plt.show()

In [ ]:
nlp.get_pipe('ner').labels